# 행정안전부 도로명주소 API
### 건물명으로부터 도로명주소 구하기

In [3]:
import requests
from urllib.parse import urlparse, quote   #quote 추가하면 '강서구청' 한글안깨짐
import pandas as pd

In [4]:
key_fd = open('roadapikey.txt', mode='r')
roadapi_key = key_fd.read(100)
key_fd.close()

In [5]:
# 행안부 개발자센터 참고 https://www.juso.go.kr/addrlink/devAddrLinkRequestGuide.do?menu=roadApi
# ?물음표 다음에 나오는 쿼리에 한글'강서구청'이 들어가면 크롬은 돌아가는데, 익스플로어는 안돌아가기도한다. 그래서 quote(인용하다)사용
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'   #행안부에정보있음
url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword=강서구청&resultType=json' #행안부 도큐보고 
urlparse(url) # .geturl()빼니까 여섯가지 정보알려줌  # '강서구청'한글이 깨지지않게

ParseResult(scheme='https', netloc='www.juso.go.kr', path='/addrlink/addrLinkApi.do', params='', query='confmKey=U01TX0FVVEgyMDIwMTExOTEwMjMzNTExMDQzNzg=&currentPage=1&countPerPage=10&keyword=강서구청&resultType=json', fragment='')

In [6]:
quote('강서구청')   #위에서 quote  추가했기때문에 인코딩됩니다.

'%EA%B0%95%EC%84%9C%EA%B5%AC%EC%B2%AD'

In [7]:
url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword={quote("강서구청")}&resultType=json'  #f'겹치니까  "강서구청"으로 변경
url

'https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey=U01TX0FVVEgyMDIwMTExOTEwMjMzNTExMDQzNzg=&currentPage=1&countPerPage=10&keyword=%EA%B0%95%EC%84%9C%EA%B5%AC%EC%B2%AD&resultType=json'

In [9]:
results = requests.get(url).json()       # **보고따라해야만하는것  geT(urlparse(url).geturl())부분을 url로 수정했다
results   # {딕셔너리타입} 'juso'부분의 'roadAddr'필요

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '6',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '',
    'engAddr': '302, Hwagok-ro, Gangseo-gu, Seoul',
    'rn': '화곡로',
    'emdNm': '화곡동',
    'zipNo': '07658',
    'roadAddrPart2': ' (화곡동)',
    'emdNo': '02',
    'sggNm': '강서구',
    'jibunAddr': '서울특별시 강서구 화곡동 980-16 강서구청',
    'siNm': '서울특별시',
    'roadAddrPart1': '서울특별시 강서구 화곡로 302',
    'bdNm': '강서구청',
    'admCd': '1150010300',
    'udrtYn': '0',
    'lnbrMnnm': '980',
    'roadAddr': '서울특별시 강서구 화곡로 302 (화곡동)',
    'lnbrSlno': '16',
    'buldMnnm': '302',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '115003005069',
    'mtYn': '0',
    'bdMgtSn': '1150010300109800016015086',
    'buldSlno': '0'},
   {'detBdNmList': '어린이집',
    'engAddr': '72, Hwagok-ro 44na-gil, Gangseo-gu, Seoul',
    'rn': '화곡로44나길',
    'emdNm': '화곡동',
    'zipNo': '07663',
    'roadAddrPart2': ' (화곡동)',
    'emdNo': '01',
    'sggNm': '강

In [10]:
addr_list = results['results']['juso']
len(addr_list)      #6개에 해당되는 빌딩이름bdNm, 도로명주소roadAddr를 가져오자

6

In [11]:
for addr in addr_list:
    print(addr['bdNm'], addr['roadAddr'])
#좌표(위도경도)를 알려면 정확한 도로명주소가 필요하다

강서구청 서울특별시 강서구 화곡로 302 (화곡동)
강서구청 서울특별시 강서구 화곡로44나길 72 (화곡동)
강서구청 부산광역시 강서구 낙동북로 477 (대저1동)
강서구청청소차고 서울특별시 강서구 남부순환로 208 (외발산동)
강서구청가양동별관 서울특별시 강서구 양천로59길 38 (가양동)
강서구청역 부산광역시 강서구 낙동북로 485 (대저1동)


### 좌표(위도경도)를 알려면 정확한 도로명주소가 필요하다
# 양천구청 부천시청 영등포구청 김포시청  5군데 한번에 알아보기;


In [12]:
bldg_list = ['강서구청', '양천구청', '부천시청', '영등포구청', '김포시청']
road_addr_list = []  #도로명주소리스트를 만들꺼니까
for bldg in bldg_list:
    url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword={quote(bldg)}&resultType=json' #quote(bldg)로변경 (위에서 선언했어서)
    urlparse(url).geturl()
    results = requests.get(url).json()   #get(url)로변경
    addr_list = results['results']['juso']
    road_addr = addr_list[0]['roadAddr']
    road_addr_list.append(road_addr)
    print(road_addr)

서울특별시 강서구 화곡로 302 (화곡동)
서울특별시 양천구 목동동로 105 (신정동)
경기도 부천시 길주로 210-2 (중동)
서울특별시 영등포구 당산로 123 (당산동3가)
경기도 김포시 사우중로 1 (사우동)


In [13]:
df = pd.DataFrame({
        'bldg': bldg_list,
        'road_addr': road_addr_list
})
df

,bldg,road_addr
0,강서구청,서울특별시 강서구 화곡로 302 (화곡동)
1,양천구청,서울특별시 양천구 목동동로 105 (신정동)
2,부천시청,경기도 부천시 길주로 210-2 (중동)
3,영등포구청,서울특별시 영등포구 당산로 123 (당산동3가)
4,김포시청,경기도 김포시 사우중로 1 (사우동)


In [14]:
df.to_csv('address.csv', sep=',', encoding='utf8', index=False)

In [ ]:
# 로드 API는 끝